In [1]:
library(data.table)
library(tidyverse)
library(fastmap)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()

In [2]:
setwd("/staging/leuven/stg_00096/home/rcools/singularity_and_git/signature.tools.lib/")

In [3]:
library(signature.tools.lib)

In [35]:
tmp_vcf<-readLines("/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/Severus_TO_2/somatic_SVs/severus_somatic.vcf")
tmp_vcf_data<-read.table("/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/Severus_TO_2/somatic_SVs/severus_somatic.vcf", stringsAsFactors = FALSE)

In [36]:
# filter for the columns names
tmp_vcf<-tmp_vcf[-(grep("#CHROM",tmp_vcf)+1):-(length(tmp_vcf))]
vcf_names<-unlist(strsplit(tmp_vcf[length(tmp_vcf)],"\t"))
names(tmp_vcf_data)<-vcf_names

head(tmp_vcf_data,1)

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,tagged_bam_D0_prefix,tagged_bam_Q1_prefix,tagged_bam_Q2_prefix,tagged_bam_Q3_prefix
,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,chr1,986975,severus_DEL4,N,<DEL>,60,PASS,PRECISE;SVTYPE=DEL;SVLEN=508;END=987483;STRANDS=+-;INSIDE_VNTR=TRUE;DETAILED_TYPE=DEL;MAPQ=60.0,GT:GQ:VAF:hVAF:DR:DV,"0/0:792:0.10:0.11,0.00,0.00:70:8","0/0:664:0.12:0.12,0.00,0.00:60:8","0/1:256:0.29:0.00,0.83,0.50:29:12","0/0:549:0.04:0.05,0.00,0.00:45:2"


In [37]:
library(dplyr)
library(tidyr)
library(stringr)

D0 <- tmp_vcf_data %>%
  select(-tagged_bam_Q2_prefix, -tagged_bam_Q3_prefix, -tagged_bam_Q1_prefix) %>%
  separate(tagged_bam_D0_prefix, into = c("GT", "GQ", "VAF_0", "hVAF", "DR", "DV"), sep = ":") %>%
  separate(INFO, into = c("V1", "SVTYPE", "SVLEN", "END", "else"), sep = ";") %>%
  mutate(
    SVTYPE = str_remove(SVTYPE, "^SVTYPE="),
    SVLEN = str_remove(SVLEN, "^SVLEN="),
    END = str_remove(END, "^END=")
  )

Q1 <- tmp_vcf_data %>%
  select(-tagged_bam_Q2_prefix, -tagged_bam_Q3_prefix, -tagged_bam_D0_prefix) %>%
  separate(tagged_bam_Q1_prefix, into = c("GT", "GQ", "VAF_0", "hVAF", "DR", "DV"), sep = ":") %>%
  separate(INFO, into = c("V1", "SVTYPE", "SVLEN", "END", "else"), sep = ";") %>%
  mutate(
    SVTYPE = str_remove(SVTYPE, "^SVTYPE="),
    SVLEN = str_remove(SVLEN, "^SVLEN="),
    END = str_remove(END, "^END=")
  )

Q2 <- tmp_vcf_data %>%
  select(-tagged_bam_Q1_prefix, -tagged_bam_Q3_prefix, -tagged_bam_D0_prefix) %>%
  separate(tagged_bam_Q2_prefix, into = c("GT", "GQ", "VAF_0", "hVAF", "DR", "DV"), sep = ":") %>%
  separate(INFO, into = c("V1", "SVTYPE", "SVLEN", "END", "else"), sep = ";") %>%
  mutate(
    SVTYPE = str_remove(SVTYPE, "^SVTYPE="),
    SVLEN = str_remove(SVLEN, "^SVLEN="),
    END = str_remove(END, "^END=")
  )

Q3 <- tmp_vcf_data %>%
  select(-tagged_bam_Q1_prefix, -tagged_bam_Q2_prefix, -tagged_bam_D0_prefix) %>%
  separate(tagged_bam_Q3_prefix, into = c("GT", "GQ", "VAF_0", "hVAF", "DR", "DV"), sep = ":") %>%
  separate(INFO, into = c("V1", "SVTYPE", "SVLEN", "END", "else"), sep = ";") %>%
  mutate(
    SVTYPE = str_remove(SVTYPE, "^SVTYPE="),
    SVLEN = str_remove(SVLEN, "^SVLEN="),
    END = str_remove(END, "^END=")
  )



Warning message:
“Expected 5 pieces. Additional pieces discarded in 1296 rows [1, 2, 3, 4, 5, 6,
7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 5 pieces. Missing pieces filled with `NA` in 48 rows [84, 94, 101,
110, 120, 137, 181, 183, 191, 208, 214, 228, 305, 322, 323, 353, 375, 446, 472,
482, ...].”
Warning message:
“Expected 5 pieces. Additional pieces discarded in 1296 rows [1, 2, 3, 4, 5, 6,
7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 5 pieces. Missing pieces filled with `NA` in 48 rows [84, 94, 101,
110, 120, 137, 181, 183, 191, 208, 214, 228, 305, 322, 323, 353, 375, 446, 472,
482, ...].”
Warning message:
“Expected 5 pieces. Additional pieces discarded in 1296 rows [1, 2, 3, 4, 5, 6,
7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 5 pieces. Missing pieces filled with `NA` in 48 rows [84, 94, 101,
110, 120, 137, 181, 183, 191, 208, 214, 228, 305, 322, 323, 353

In [ ]:
head(Q2,15)

In [39]:
chr <- c(paste0("chr", 1:22), "chrX")

filtered_list<- list(D0,Q1,Q2,Q3)

Q1$CHROM <- Q1$'#CHROM'
D0$CHROM <- D0$'#CHROM'
Q2$CHROM <- Q2$'#CHROM'
Q3$CHROM <- Q3$'#CHROM'

In [40]:
setwd("/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/Severus_TO_2/somatic_SVs/plots")

In [41]:
process_svtype <- function(df) {
  
  df <- df %>% filter(GT != "0")  # Filter rows where GT is not "0"
  
  # Handle BND rows
  bnd_df <- df %>% 
    filter(SVTYPE == "BND") %>%
    mutate(
      ALT_clean = ALT %>% 
        str_replace_all("N", "") %>% 
        str_replace_all("\\[", "") %>% 
        str_replace_all("\\]", "")
    ) %>%
    separate(ALT_clean, into = c("chrom2", "start2"), sep = ":") %>%
    mutate(
      svclass = "translocation",
      chrom1 = CHROM,
      start1 = as.numeric(POS),
      start2 = as.numeric(start2),
      end1 = start1 + 1,
      end2 = start2 + 1
    ) %>%
    select(chrom1, start1, end1, chrom2, start2, end2, svclass)
  
  # Handle other SVTYPEs
  other_df <- df %>% 
    filter(SVTYPE != "BND") %>%
    mutate(
      chrom1 = CHROM,
      start1 = as.numeric(POS),
      chrom2 = CHROM,
      start2 = case_when(
        SVTYPE == "DEL" ~ as.numeric(END),
        SVTYPE == "INS" ~ as.numeric(POS) + as.numeric(SVLEN),
        SVTYPE %in% c("DUP", "INV") ~ as.numeric(END),
        TRUE ~ NA_real_
      ),
      end1 = start1 + 1,
      end2 = start2 + 1,
      svclass = case_when(
        SVTYPE == "DEL" ~ "DEL",
        SVTYPE == "INS" ~ "INS",
        SVTYPE == "DUP" ~ "DUP",
        SVTYPE == "INV" ~ "INV",
        TRUE ~ NA_character_
      )
    ) %>%
    select(chrom1, start1, end1, chrom2, start2, end2, svclass) %>%
    filter(!is.na(svclass))
  
  bind_rows(bnd_df, other_df)
}

  

# change for every sample

In [111]:
##################
################
test <- process_svtype(Q3)
sample <-"Q3"

Warning message:
“There were 2 warnings in `mutate()`.
The first warning was:
ℹ In argument: `start2 = case_when(...)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning.”


#Change for every sample:

In [112]:
nrow(test)

[1] 1184

In [113]:
tail(test)

,chrom1,start1,end1,chrom2,start2,end2,svclass
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
1179,chrX,130088172,130088173,chrX,130088658,130088659,DEL
1180,chrX,134905216,134905217,chrX,134906330,134906331,DEL
1181,chrX,136948857,136948858,chrX,136950072,136950073,DEL
1182,chrX,148314271,148314272,chrX,148315125,148315126,DEL
1183,chrX,154391454,154391455,chrX,154391871,154391872,DEL
1184,chrX,154795662,154795663,chrX,154796946,154796947,DEL


In [114]:
valid_chroms <- paste0("chr", c(1:22, "X", "Y"))

# Filter sv_bedpe to include only links between valid chromosomes
test <- test %>%
  dplyr::filter(chrom1 %in% valid_chroms & chrom2 %in% valid_chroms)

In [115]:
unique(test$chrom2)

[1] "chr2"  "chr3"  "chr8"  "chr14" "chr5"  "chr7"  "chr9"  "chr10" "chr11"
[10] "chr17" "chr12" "chr16" "chrX"  "chr20" "chr1"  "chr4"  "chr6"  "chr13"
[19] "chr15" "chr18" "chr19" "chr21" "chr22"

In [116]:
test$sample <- sample

In [117]:
test <- test %>% 
  filter(!(svclass == "translocation" & chrom1 == chrom2))

In [119]:
write.table(test, 
            file = paste0("output_",sample,"_filter.bed"), 
            sep = "\t", 
            quote = FALSE, 
            row.names = FALSE, 
            col.names = TRUE)

In [120]:
#Copy number file
ascat <- fread(paste0("/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/ASCAT_replot/",sample,"/",sample,".segments.txt"))

In [121]:
head(ascat)

sample,chr,startpos,endpos,nMajor,nMinor
<chr>,<chr>,<int>,<int>,<int>,<int>
Q3,1,809641,54073027,1,1
Q3,1,54077129,54859815,1,0
Q3,1,54860467,55023987,0,0
Q3,1,55034267,83757485,1,1
Q3,1,83760556,84677969,1,0
Q3,1,84681961,94739712,1,1


In [122]:
#Plus 65 for plottig purposes -> so we plot gains woth the najor allele without a gradient
ascat$total.copy.number.inNormal <- 2
ascat$minor.copy.number.inNormal <- 1
ascat$minor.copy.number.inTumour <- ascat$nMinor
ascat$total.copy.number.inTumour <- ifelse(
  ascat$nMajor == 1 |ascat$nMajor == 0 ,
  ascat$nMajor + ascat$nMinor,
  65
)
ascat$Chromosome <- ascat$chr
ascat$chromStart <- ascat$startpos
ascat$chromEnd <- ascat$endpos
ascat$seg_no <- row.names(ascat)


In [123]:
head(ascat)

sample,chr,startpos,endpos,nMajor,nMinor,total.copy.number.inNormal,minor.copy.number.inNormal,minor.copy.number.inTumour,total.copy.number.inTumour,Chromosome,chromStart,chromEnd,seg_no
<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<chr>,<int>,<int>,<chr>
Q3,1,809641,54073027,1,1,2,1,1,2,1,809641,54073027,1
Q3,1,54077129,54859815,1,0,2,1,0,1,1,54077129,54859815,2
Q3,1,54860467,55023987,0,0,2,1,0,0,1,54860467,55023987,3
Q3,1,55034267,83757485,1,1,2,1,1,2,1,55034267,83757485,4
Q3,1,83760556,84677969,1,0,2,1,0,1,1,83760556,84677969,5
Q3,1,84681961,94739712,1,1,2,1,1,2,1,84681961,94739712,6


In [124]:
write.table(ascat, 
            file = paste0("ascat_",sample,".txt"), 
            sep = "\t", 
            quote = FALSE, 
            row.names = FALSE, 
            col.names = TRUE)

In [125]:
tmp_vcf_data<-read.table(paste0("/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/CLAIRS_TO/coverage_10/",sample,"/filtered.vcf.gz"), stringsAsFactors = FALSE)
tmp_vcf<-readLines(paste0("/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/CLAIRS_TO/coverage_10/",sample,"/filtered.vcf.gz"))

# filter for the columns names
tmp_vcf<-tmp_vcf[-(grep("#CHROM",tmp_vcf)+1):-(length(tmp_vcf))]
vcf_names<-unlist(strsplit(tmp_vcf[length(tmp_vcf)],"\t"))
names(tmp_vcf_data)<-vcf_names

head(tmp_vcf_data,1)

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Q3
,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,chr1,617409,.,A,C,20.6958,PASS,FAU=0;FCU=6;FGU=0;FTU=0;RAU=0;RCU=8;RGU=0;RTU=0;SB=1,GT:GQ:DP:AF:AD:AU:CU:GU:TU,"1/1:20:14:1:0,14:0:14:0:0"


In [126]:
tmp_vcf_data$chrom <- tmp_vcf_data$`#CHROM`

In [127]:
snv <- data.frame( 
  chr = tmp_vcf_data$chrom, 
  position = tmp_vcf_data$POS, 
  REF = tmp_vcf_data$REF, 
  ALT = tmp_vcf_data$ALT
)


In [128]:
write.table(snv, 
            file = paste0("snv_",sample,".txt"), 
            sep = "\t", 
            quote = FALSE, 
            row.names = FALSE, 
            col.names = TRUE)

In [129]:
getwd()

[1] "/lustre1/project/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/Severus_TO_2/somatic_SVs/plots"

In [130]:
getwd()

[1] "/lustre1/project/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/Severus_TO_2/somatic_SVs/plots"

In [131]:
snv_file <- paste0("/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/CLAIRS_TO/coverage_10/",sample,"/filtered.vcf.gz")
ascat <- paste0("/lustre1/project/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/Severus_TO_2/somatic_SVs/plots/ascat_",sample,".txt")
SV <- paste0("/lustre1/project/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/Severus_TO_2/somatic_SVs/plots/output_",sample,"_filter.bed")

In [ ]:
genomeChart(outfilename = paste0(sample,"_circos.pdf"), sample_name = sample, SNV_vcf_file = snv_file, CNV_tab_file=ascat,SV_bedpe_file=SV,runKataegis=FALSE,genome.v = "hg38", Indels_vcf_file=NULL, Indels_tab_file=NULL)

In [133]:
getwd()

[1] "/lustre1/project/stg_00096/home/projects/2023_Cools_B-ALL/FINAL_DNA_ANALYSIS/new_Variant_calling/Severus_TO_2/somatic_SVs/plots"